In [11]:
import straxen

# Test Plugin implementation
Thanks to the URLConfig, to switch to CMT2.0 all you have to do is change the URL in your context config or change the default value in the plugin definition:
As an example lets load the tpc pmt gains and store the average and stdv values in the events data just for fun.

```python

class CMT2Example(strax.Plugin):
    """
    Plugin which merges the information of all event data_kinds into a
    single data_type.
    """

    tpc_gains = URLConfig(default='cmt2://pmt_gains'
                                  '?version=v6'
                                  '&detector=tpc'
                                  '&run_id=plugin.run_id'
                                  '&attr=value'
                                  '&sort=pmt',
                        cache=True)

    depends_on = ('event_basics',)

    save_when = strax.SaveWhen.NEVER

    provides = 'gain_stats'

    dtype = strax.time_fields + [
        ('gain_average', np.float64, 'Average tpc gains'),
        ('gain_std', np.float64, 'stdv of tpc gains'),
        ]

    __version__ = '0.0.1'


    def compute(self, events):
        gains = self.tpc_gains
        result = np.zeros(len(events), dtype=self.dtype)
        result['time'] = events['time']
        result['endtime'] = events['endtime']
        result['gain_average'] = np.average(gains)
        result['gain_std'] = np.std(gains)
       
        return  result



```

# Using the CMT2.0 test plugin

## Register the plugin

In [2]:
st = straxen.test_utils.nt_test_context()
st.register(straxen.CMT2Example)

straxen.plugins.cmt2_example.CMT2Example

## Get the plugin

In [3]:
p = st.get_single_plugin('012882', 'gain_stats')

In [4]:
p.config

{'tpc_gains': 'cmt2://pmt_gains?version=v6&detector=tpc&run_id=plugin.run_id&attr=value&sort=pmt'}

## Make the test data

In [6]:
df = st.get_df('012882', 'gain_stats')

Loading gain_stats: |          | 0.00 % [00:00<?]

## Inspect the test data
wow, so beautiful

In [47]:
df.head()

,time,endtime,gain_average,gain_std
0,124900000,125756920,0.010926,0.005491
1,373103602,375986430,0.010926,0.005491
2,622431032,625294240,0.010926,0.005491
3,872454492,875318350,0.010926,0.005491
4,1123095132,1125970040,0.010926,0.005491


## Inspect the gains array
wow, so pretty

In [9]:
p.tpc_gains[:10]

[0.007978886026682173,
 0.008737005473622564,
 0.011696056533065597,
 0.013830639629634581,
 0.008746130220955254,
 0.026788049778946853,
 0.008659173466360496,
 0.007938550091345057,
 0.008616717894222567,
 0.00921492158937479]